In [2]:
import json
import re
from typing import Dict, List, Optional, Union
from dataclasses import dataclass
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np


@dataclass
class CFGSimilarityResult:
    """存储CFG比较结果的数据类"""
    filename: str
    edge_coverage: float
    content_similarity: float
    structure_similarity: float
    matched_edges: int
    gt_edges: int
    llm_edges: int
    nested_results: Optional[Dict[str, 'CFGSimilarityResult']] = None


class CFGComparator:
    def __init__(self):
        """初始化CFG比较器"""
        pass

    @staticmethod
    def count_edges(cfg: Dict) -> int:
        """
        递归计算CFG中的边数量，包括所有嵌套结构的边
        """
        def count_edges_recursive(blocks):
            edge_count = 0
            for block in blocks:
                successors = block.get("successors", [])
                edge_count += len(successors)  # 当前 block 的边数量
                # 递归计算 successors 的边
                for successor in successors:
                    if isinstance(successor, dict):  # successor 也是一个带有 blocks 的嵌套结构
                        edge_count += count_edges_recursive(successor.get("blocks", []))
            return edge_count

        # 统计顶层 blocks 的边
        edge_count = count_edges_recursive(cfg.get("blocks", []))

        # 递归统计嵌套函数和类的边
        for func in cfg.get("functions", []):
            edge_count += CFGComparator.count_edges(func)
        for cls in cfg.get("classes", []):
            edge_count += CFGComparator.count_edges(cls)

        return edge_count

    def structure_similarity(self, llm_cfg: Dict, static_cfg: Dict) -> float:
        """
        计算两个CFG的结构相似度（基于边的数量）。
        """
        llm_edges = self.count_edges(llm_cfg)
        static_edges = self.count_edges(static_cfg)

        # 如果两者都没有边，则结构上可以视为“极度简化”——按需设为 1.0
        if llm_edges == 0 and static_edges == 0:
            return 1.0
        if llm_edges == 0 or static_edges == 0:
            return 0.0

        # 较简单的衡量：用最小/最大，值越接近1越表示边数接近
        return min(llm_edges, static_edges) / max(llm_edges, static_edges)

    def content_similarity(self, llm_cfg: Dict, static_cfg: Dict) -> float:
        """
        计算两个CFG的内容相似度（基于 blocks 的内容）。
        如果 label 是列表，则将其合并为一个字符串。
        并在向量化前检查文本是否为空。
        """
        llm_blocks = llm_cfg.get("blocks", [])
        static_blocks = static_cfg.get("blocks", [])

        # 如果两边都没有 blocks，视为内容相同
        if not llm_blocks and not static_blocks:
            return 1.0
        # 如果一边没 blocks 则视为没有可比内容
        if not llm_blocks or not static_blocks:
            return 0.0

        def unify_label(label_value: Union[str, List[str], None]) -> str:
            """将 label 统一成单一字符串。"""
            if isinstance(label_value, list):
                return "\n".join(str(item) for item in label_value)
            elif isinstance(label_value, str):
                return label_value
            else:
                return ""

        # 将 blocks 的 label 统一转换为字符串后再拼成大文本
        llm_code = "\n".join(unify_label(block.get("label", "")) for block in llm_blocks)
        static_code = "\n".join(unify_label(block.get("label", "")) for block in static_blocks)

        # 进一步清洗空白字符
        llm_code = llm_code.strip()
        static_code = static_code.strip()

        # 如果两个文本都空，视为相似度 1.0
        if not llm_code and not static_code:
            return 1.0
        # 如果只有一方空，则相似度 0.0
        if not llm_code or not static_code:
            return 0.0

        # ======== 以下为正常文本相似度计算 ========
        def custom_tokenizer(text):
            return re.findall(r'\b\w+\b', text)

        def custom_preprocessor(text):
            # 简单删除 # 开头的注释，可按需扩展
            text = re.sub(r'#.*', '', text)
            return text

        vectorizer = CountVectorizer(
            tokenizer=custom_tokenizer,
            preprocessor=custom_preprocessor,
            stop_words='english',
            ngram_range=(1, 2)
        )

        try:
            vectors = vectorizer.fit_transform([llm_code, static_code])
            similarity = cosine_similarity(vectors[0], vectors[1])[0][0]
        except ValueError:
            # 捕获 "empty vocabulary" 异常，如果文本过于短或只含停用词，fallback 为 0.0
            similarity = 0.0

        return similarity

    def compare_cfgs(self, llm_cfg: Dict, static_cfg: Dict, name: str) -> CFGSimilarityResult:
        """递归比较两个CFG并返回相似度结果"""
        structure_sim = self.structure_similarity(llm_cfg, static_cfg)
        content_sim = self.content_similarity(llm_cfg, static_cfg)

        gt_edges = self.count_edges(static_cfg)
        llm_edges = self.count_edges(llm_cfg)

        matched_edges = int(structure_sim * min(gt_edges, llm_edges))
        edge_coverage = matched_edges / gt_edges if gt_edges > 0 else 0

        nested_results = {}

        # === 递归比较 methods (functions) ===
        llm_functions = {f["name"]: f for f in llm_cfg.get("functions", [])}
        static_functions = {f["name"]: f for f in static_cfg.get("functions", [])}
        common_functions = set(llm_functions.keys()) & set(static_functions.keys())

        for func_name in common_functions:
            nested_results[f"function_{func_name}"] = self.compare_cfgs(
                llm_functions[func_name],
                static_functions[func_name],
                func_name
            )

        # === 递归比较 classes ===
        llm_classes = {c["name"]: c for c in llm_cfg.get("classes", [])}
        static_classes = {c["name"]: c for c in static_cfg.get("classes", [])}
        common_classes = set(llm_classes.keys()) & set(static_classes.keys())

        for class_name in common_classes:
            nested_results[f"class_{class_name}"] = self.compare_cfgs(
                llm_classes[class_name],
                static_classes[class_name],
                class_name
            )

        return CFGSimilarityResult(
            filename=name,
            edge_coverage=edge_coverage,
            content_similarity=content_sim,
            structure_similarity=structure_sim,
            matched_edges=matched_edges,
            gt_edges=gt_edges,
            llm_edges=llm_edges,
            nested_results=nested_results if nested_results else None
        )


class CFGEvaluator:
    def __init__(self, llm_cfg_dir: str, static_cfg_dir: str, result_file: str):
        self.llm_cfg_dir = Path(llm_cfg_dir)
        self.static_cfg_dir = Path(static_cfg_dir)
        self.result_file = Path(result_file)
        self.comparator = CFGComparator()
        self.results: List[CFGSimilarityResult] = []

    def process_file(self, llm_cfg_path: Path) -> Optional[CFGSimilarityResult]:
        static_cfg_path = self.static_cfg_dir / llm_cfg_path.name
        if not static_cfg_path.exists():
            return None

        with open(llm_cfg_path, "r", encoding="utf-8") as f:
            llm_cfg = json.load(f)
        with open(static_cfg_path, "r", encoding="utf-8") as f:
            static_cfg = json.load(f)

        result = self.comparator.compare_cfgs(llm_cfg, static_cfg, llm_cfg_path.name)
        self.results.append(result)
        self.save_results()
        return result

    def save_results(self):
        with open(self.result_file, "w", encoding="utf-8") as f:
            json.dump(
                [self._result_to_dict(r) for r in self.results],
                f,
                indent=2,
                ensure_ascii=False
            )

    def evaluate_all(self) -> List[CFGSimilarityResult]:
        llm_cfg_paths = list(self.llm_cfg_dir.glob("*.json"))
        for llm_cfg_path in tqdm(llm_cfg_paths, desc="处理CFG文件"):
            self.process_file(llm_cfg_path)
        return self.results

    @staticmethod
    def _result_to_dict(result: CFGSimilarityResult) -> Dict:
        return {
            "filename": result.filename,
            "edge_coverage": result.edge_coverage,
            "content_similarity": result.content_similarity,
            "structure_similarity": result.structure_similarity,
            "matched_edges": result.matched_edges,
            "gt_edges": result.gt_edges,
            "llm_edges": result.llm_edges,
            "nested_results": (
                {
                    k: CFGEvaluator._result_to_dict(v)
                    for k, v in result.nested_results.items()
                }
                if result.nested_results
                else None
            )
        }


def calculate_aggregate_metrics(results: List[CFGSimilarityResult]) -> Dict[str, float]:
    """对比较结果做全局统计。"""
    if not results:
        return {
            "total_cfgs_compared": 0,
            "average_edge_coverage": 0.0,
            "average_content_similarity": 0.0,
            "average_structure_similarity": 0.0,
            "total_gt_edges": 0,
            "total_llm_edges": 0,
            "total_matched_edges": 0
        }

    metrics = {
        "total_cfgs_compared": len(results),
        "average_edge_coverage": float(np.mean([r.edge_coverage for r in results])),
        "average_content_similarity": float(np.mean([r.content_similarity for r in results])),
        "average_structure_similarity": float(np.mean([r.structure_similarity for r in results])),
        "total_gt_edges": sum(r.gt_edges for r in results),
        "total_llm_edges": sum(r.llm_edges for r in results),
        "total_matched_edges": sum(r.matched_edges for r in results)
    }
    return metrics


def main():
    evaluator = CFGEvaluator(
        llm_cfg_dir="merged_llm_cfg_with_line_no",
        static_cfg_dir="../../dataset/python_cfg",
        result_file="evaluation_results.json"
    )

    results = evaluator.evaluate_all()
    metrics = calculate_aggregate_metrics(results)

    print("\nAutomatic Evaluation Summary:")
    print(f"Total CFGs compared: {metrics['total_cfgs_compared']}")
    print(f"Average Edge Coverage: {metrics['average_edge_coverage']:.2f}")
    print(f"Average Content Similarity: {metrics['average_content_similarity']:.2f}")
    print(f"Average Structure Similarity: {metrics['average_structure_similarity']:.2f}")

    print(f"Total GT edges: {metrics['total_gt_edges']}")
    print(f"Total LLM edges: {metrics['total_llm_edges']}")
    print(f"Total Matched edges: {metrics['total_matched_edges']}")
    print(f"Precision: {metrics['total_matched_edges'] / metrics['total_llm_edges']}")
    print(f"Recall: {metrics['total_matched_edges'] / metrics['total_gt_edges']}")
    print(f"F1 Score: {2 * metrics['total_matched_edges'] / (metrics['total_llm_edges'] + metrics['total_gt_edges'])}")

    with open("evaluation_metrics.json", "w", encoding="utf-8") as f:
        json.dump(metrics, f, indent=2, ensure_ascii=False)


if __name__ == "__main__":
    main()


处理CFG文件:   0%|          | 0/190 [00:00<?, ?it/s]/home/miniconda3/envs/llm_analysis/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
处理CFG文件: 100%|██████████| 190/190 [00:06<00:00, 27.35it/s]


Automatic Evaluation Summary:
Total CFGs compared: 190
Average Edge Coverage: 0.54
Average Content Similarity: 0.76
Average Structure Similarity: 0.71
Total GT edges: 856
Total LLM edges: 938
Total Matched edges: 570
Precision: 0.6076759061833689
Recall: 0.6658878504672897
F1 Score: 0.6354515050167224


In [28]:
import json
import os
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import cpu_count
from tqdm import tqdm
# 假设 get_llm_answers 从您的 llm 模块导入
from llm import get_llm_answers  

from typing import Set, Tuple

def convert_cfg_json_to_edges(cfg_json: dict) -> list[str]:
    """基于代码块内容生成控制流边，不依赖节点ID"""
    edges = []
    edge_counter = 0

    def get_block_label(block: dict) -> str:
        """提取标准化的块标签"""
        label = block.get('label', '')
        # 确保label是字符串类型
        if isinstance(label, list):
            label = ' '.join(label)
        elif not isinstance(label, str):
            label = str(label)
        # 清理换行和多余空格
        return ' '.join(label.replace('\n', '\\n').strip().split())

    def process_entity(entity: dict):
        """递归处理各类实体（函数/类/块）"""
        # 处理当前实体的直接blocks
        for block in entity.get('blocks', []):
            process_block(block)

        # 递归处理嵌套结构
        for key in ['functions', 'classes']:
            for sub_entity in entity.get(key, []):
                process_entity(sub_entity)

    def process_block(block: dict):
        """处理单个代码块及其后继"""
        nonlocal edge_counter
        
        # 获取当前块的规范化标签
        source_label = get_block_label(block)
        if not source_label:
            return

        # 处理直接后继
        for succ in block.get('successors', []):
            target_label = get_block_label(succ)
            if target_label:
                edges.append(
                    f"Edge {edge_counter}: [Source] {source_label} => "
                    f"[Target] {target_label}"
                )
                edge_counter += 1

        # 递归处理嵌套blocks（如if/else内的块）
        for key in ['blocks', 'successors']:
            for sub_block in block.get(key, []):
                process_block(sub_block)

    # 从根节点开始处理
    process_entity(cfg_json)
    return edges

def get_prompt(code: str, llm_cfg: dict, static_cfg: dict):
    prompt = f"""
Role: Control Flow Graph Validation Specialist
Objective: Accurately compare CFG structures between static analysis (ground truth) and LLM generation

### JSON Structure Definition
Ground Truth (static_cfg) & LLM Output (llm_cfg) follow:
[
    "Edge 0: [Source] node_A -> [Target] node_B",
    "Edge 1: [Source] node_C -> [Target] node_D",
    ...
]

### Comparison Criteria
1. Structure Matching:
   Match edges when:
   - Same branching pattern (sequential/conditional/loop)
   - Equivalent depth in nested structure
   - Matching control flow order

2. Mismatch Conditions:
   - Different number of successors in equivalent blocks
   - Inconsistent branch types (e.g., true/false vs multiple)
   - Missing/extra exception handling flows

### Analysis Task
1. For static_cfg:
   - Count total edges (ground truth)
   - Map control flow patterns

2. For llm_cfg:
   - Count total generated edges
   - Identify structurally matched edges
   - Traverse each edge and check if it corresponds to static analysis

3. Output (JSON):
{{
  "edge_analysis": {{
    "static_total": "Number of edges from static analysis",
    "llm_total": "Number of edges generated by LLM",
    "matched_edges": {{
      "exact_matches": "Number of exactly matched edges (type + position)", 
      "partial_matches": "Number of type-matched edges with different positions"
    }},
    "accuracy_metrics": {{
      "precision": "exact_matches / llm_total",
      "recall": "exact_matches / static_total", 
      "f1_score": "2*(precision*recall)/(precision+recall)"
    }}
  }},
  "structure_validation": {{
    "missing_blocks": ["Unmatched static block IDs"],
    "extra_blocks": ["Extra LLM block IDs"]
  }}
}}

### Input Data
TypeScript Code:
{code}

Static Analysis CFG (Ground Truth):
{json.dumps(convert_cfg_json_to_edges(static_cfg), indent=2)}

LLM Generated CFG:
{json.dumps(convert_cfg_json_to_edges(llm_cfg), indent=2)}

Output JSON analysis ONLY.
"""
    return prompt.strip()
### Output Schema
# {{
#   "total_gt_edges": "Count of all DOT edges",
#   "total_llm_edges": "Count of all JSON edges",
#   "total_matched_edges": "Structurally aligned edges"
# }}

def process_file(i):
    """处理单个文件的CFG对比"""
    # 路径配置
    code_path = f"../../dataset/python/{i}.py"
    llm_cfg_path = f"llm_cfg_with_line_no/{i}.json"
    static_cfg_path = f"../../dataset/python_cfg/{i}.json"  # 改为JSON路径
    result_path = f"results/{i}.json"

    # 跳过已处理文件
    if os.path.exists(result_path):
        return

    # 校验文件存在性
    missing_files = [
        p for p in [code_path, llm_cfg_path, static_cfg_path]
        if not os.path.exists(p)
    ]
    if missing_files:
        print(f"Missing files for index {i}: {missing_files}")
        return

    try:
        # 读取并解析输入数据
        code = open(code_path).read()
        with open(llm_cfg_path) as f:
            llm_cfg = json.load(f)
        with open(static_cfg_path) as f:  # 读取JSON格式的static_cfg
            static_cfg = json.load(f)

        # 生成prompt并获取结果
        prompt = get_prompt(code, llm_cfg, static_cfg)
        # print(prompt)
        result = json.loads(get_llm_answers(prompt, model_name="gpt-4o", require_json=True))

        #print(json.dumps(result, indent=2))
        
        # 保存结果
        with open(result_path, "w") as f:
            json.dump(result, f, indent=2)

    except json.JSONDecodeError as e:
        print(f"JSON解析失败 {i}: {str(e)}")
    except Exception as e:
        print(f"处理文件 {i} 时出错: {str(e)}")

def main():
    # 初始化结果目录
    os.makedirs("results", exist_ok=True)

    # process_file(0)
    
    # 创建任务列表
    file_indices = list(range(200))  # 根据实际文件数量调整
    
    # 并行处理
    with ThreadPoolExecutor(max_workers=cpu_count()) as executor:
        tasks = executor.map(process_file, file_indices)
        list(tqdm(tasks, total=len(file_indices), desc="Processing CFGs"))

if __name__ == "__main__":
    main()

Missing files for index 97: ['../../dataset/python/97.py', 'llm_cfg_with_line_no/97.json', '../../dataset/python_cfg/97.json']
Missing files for index 101: ['llm_cfg_with_line_no/101.json']
Missing files for index 120: ['llm_cfg_with_line_no/120.json']
Missing files for index 133: ['llm_cfg_with_line_no/133.json']


Processing CFGs: 100%|██████████| 200/200 [00:56<00:00,  3.55it/s]

JSON解析失败 186: Unterminated string starting at: line 64 column 7 (char 13675)


In [29]:
import os
import json
import numpy as np

def calculate_global_metrics(results_dir: str) -> dict:
    """精确的全局指标统计"""
    metrics = {
        "total_files": 0,
        "gt_edges": 0,
        "llm_edges": 0,
        "exact_matches": 0,
        "partial_matches": 0,
        "missing_blocks": set(),
        "extra_blocks": set(),
        "file_errors": []
    }

    # 遍历结果目录
    for filename in os.listdir(results_dir):
        if not filename.endswith(".json"):
            continue

        filepath = os.path.join(results_dir, filename)
        try:
            with open(filepath) as f:
                data = json.load(f)

            # 核心指标累加
            edge_analysis = data["edge_analysis"]
            metrics["gt_edges"] += edge_analysis["static_total"]
            metrics["llm_edges"] += edge_analysis["llm_total"]
            metrics["exact_matches"] += edge_analysis["matched_edges"]["exact_matches"]
            metrics["partial_matches"] += edge_analysis["matched_edges"]["partial_matches"]

            # 结构验证统计
            structure = data["structure_validation"]
            metrics["missing_blocks"].update(map(str, structure["missing_blocks"]))
            metrics["extra_blocks"].update(map(str, structure["extra_blocks"]))

            metrics["total_files"] += 1

        except Exception as e:
            metrics["file_errors"].append(f"{filename}: {str(e)}")
            continue

    # 计算衍生指标
    total_matched = metrics["exact_matches"] + metrics["partial_matches"]
    
    precision = total_matched / metrics["llm_edges"] if metrics["llm_edges"] > 0 else 0
    recall = total_matched / metrics["gt_edges"] if metrics["gt_edges"] > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return {
        "file_count": metrics["total_files"],
        "edge_metrics": {
            "static_total": metrics["gt_edges"],
            "llm_total": metrics["llm_edges"],
            "exact_matches": metrics["exact_matches"],
            "partial_matches": metrics["partial_matches"],
            "total_matched": total_matched,
            "precision": round(precision, 4),
            "recall": round(recall, 4),
            "f1_score": round(f1, 4)
        },
        "structure_metrics": {
            "missing_blocks_count": len(metrics["missing_blocks"]),
            "extra_blocks_count": len(metrics["extra_blocks"]),
            "missing_block_samples": list(metrics["missing_blocks"])[:5],  # 示例显示前5个
            "extra_block_samples": list(metrics["extra_blocks"])[:5]
        },
        "data_quality": {
            "error_files": len(metrics["file_errors"]),
            "error_samples": metrics["file_errors"][:3]  # 示例显示前3个错误
        }
    }

def print_report(metrics: dict):
    """格式化输出报告"""
    print("CFG 评估报告")
    print("=" * 50)
    print(f"分析文件总数: {metrics['file_count']}")
    
    print("\n[边匹配分析]")
    em = metrics['edge_metrics']
    print(f"静态分析总边数: {em['static_total']}")
    print(f"LLM生成总边数: {em['llm_total']}")
    print(f"精确匹配边数: {em['exact_matches']} ({em['exact_matches']/em['static_total']:.2%})")
    print(f"部分匹配边数: {em['partial_matches']} ({em['partial_matches']/em['static_total']:.2%})")
    print(f"总匹配边数: {em['total_matched']} ({em['total_matched']/em['static_total']:.2%})")
    print(f"精确率 (Precision): {em['precision']:.4f}")
    print(f"召回率 (Recall): {em['recall']:.4f}")
    print(f"F1 值: {em['f1_score']:.4f}")

    print("\n[结构验证]")
    sm = metrics['structure_metrics']
    print(f"缺失块总数: {sm['missing_blocks_count']} (示例: {', '.join(sm['missing_block_samples'])})")
    print(f"多余块总数: {sm['extra_blocks_count']} (示例: {', '.join(sm['extra_block_samples'])})")

    if metrics['data_quality']['error_files'] > 0:
        print("\n[数据质量问题]")
        print(f"错误文件数: {metrics['data_quality']['error_files']}")
        print("示例错误:")
        for err in metrics['data_quality']['error_samples']:
            print(f"  - {err}")

    print("=" * 50)

# 使用示例
results_dir = "results"
metrics = calculate_global_metrics(results_dir)
print_report(metrics)

CFG 评估报告
分析文件总数: 193

[边匹配分析]
静态分析总边数: 2919
LLM生成总边数: 3088
精确匹配边数: 1388 (47.55%)
部分匹配边数: 492 (16.86%)
总匹配边数: 1880 (64.41%)
精确率 (Precision): 0.6088
召回率 (Recall): 0.6441
F1 值: 0.6259

[结构验证]
缺失块总数: 941 (示例: #90\nassert len(request.categories) == 2, Edge 4: [Source] #30\nconnection_manager.subscriptions['test_graph'] = {mock_websocket}\nawait connection_manager.unsubscribe('test_graph', mock_websocket) => [Target] #31\nassert 'test_graph' not in connection_manager.subscriptions, Edge 2: [Source] #3\n"""\n Generate valid combinations of balanced parentheses using recursion.\n\n :param partial: A string representing the current combination.\n :param open_count: An integer representing the count of open parentheses.\n :param close_count: An integer representing the count of close parentheses.\n :param n: An integer representing the total number of pairs.\n :param result: A list to store valid combinations.\n :return: None\n\n This function uses recursion to explore all possible combinations,